In [ ]:
"""
!pip install kaggle

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle competitions download -c digit-recognizer 

!mkdir ../input/
!mkdir ../input/digit-recognizer/

import zipfile
zip_ref = zipfile.ZipFile('train.csv.zip', 'r')
zip_ref.extractall('../input/digit-recognizer/')
zip_ref.close()

zip_ref = zipfile.ZipFile('test.csv.zip', 'r')
zip_ref.extractall('../input/digit-recognizer/')
zip_ref.close()
"""

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import itertools

from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import RMSprop,Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau,EarlyStopping
import tensorflow

random_seed = 7
np.random.seed(random_seed)
tensorflow.random.set_seed(random_seed)

In [ ]:
# Load the data
train = pd.read_csv("../input/digit-recognizer/train.csv")
test = pd.read_csv("../input/digit-recognizer/test.csv")

In [ ]:
train

In [ ]:
test

In [ ]:
num_classes = len(train["label"].unique())
Y_train = train["label"]
X_train = train.drop(labels = ["label"],axis = 1) 
Y_train.value_counts()

In [ ]:
# Normalize the data
X_train = X_train / 255
test = test / 255

In [ ]:
# Reshape image in 3 dimensions (height = 28px, width = 28px , canal = 1)
X_train = X_train.values.reshape(-1,28,28,1)
test = test.values.reshape(-1,28,28,1)
X_train.shape

In [ ]:
# Encode labels to one hot vectors (ex : 2 -> [0,0,1,0,0,0,0,0,0,0])
Y_train = to_categorical(Y_train, num_classes = num_classes)
Y_train[0]

In [ ]:
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size = 0.05, random_state=random_seed)

In [ ]:
plt.imshow(X_train[7][:,:,0],cmap="gray")

In [ ]:
input_shape = X_train.shape[1:]
input_shape

In [ ]:
def createModel(input_shape=(28, 28, 1)):
    model = Sequential()

    model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                     activation ='relu', input_shape = input_shape))
    model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                     activation ='relu'))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.25))


    model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                     activation ='relu'))
    model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                     activation ='relu'))
    model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
    model.add(Dropout(0.25))


    model.add(Flatten())
    model.add(Dense(256, activation = "relu"))
    model.add(Dropout(0.5))
    model.add(Dense(10, activation = "softmax"))
    return model

In [ ]:
# Compile the model
model = createModel(input_shape)

reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)
earlystopper = EarlyStopping(patience=15, verbose=1, monitor='val_accuracy',restore_best_weights=True)
epochs = 100 
batch_size = 128

In [ ]:
for i in range(2):
    model.compile(optimizer = Adam(1e-3) , loss = "categorical_crossentropy", metrics=["accuracy"])
    history = model.fit(X_train, Y_train, batch_size = batch_size, epochs = epochs, 
                    validation_data = (X_val, Y_val), verbose = 1,
                    callbacks=[reduce_lr,earlystopper])
    print("\n")

In [ ]:
model.evaluate(X_val, Y_val, verbose=1)

In [ ]:
# With data augmentation to prevent overfitting 

datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=5,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.05, # Randomly zoom image 
        width_shift_range=0.05,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.05,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images


datagen.fit(X_train)

In [ ]:
examples = 8
dataGenerate = datagen.flow(X_train,Y_train, batch_size=examples),
fig, axs = plt.subplots(1, examples, figsize=(25, 3)) 
i=0
for chunk in dataGenerate[0][0][0]:
    axs[i].imshow((chunk*255).astype('uint8'),cmap="gray")
    i+=1
plt.show() 

In [ ]:
for i in range(2):
    model.compile(optimizer = Adam(1e-3) , loss = "categorical_crossentropy", metrics=["accuracy"])
    # Fit the model
    history = model.fit(datagen.flow(X_train,Y_train, batch_size=batch_size),
                                  epochs = epochs, validation_data = (X_val,Y_val),
                                  verbose = 1, steps_per_epoch=X_train.shape[0] // batch_size,
                                  callbacks=[reduce_lr,earlystopper])
    print("\n")

In [ ]:
model.evaluate(X_val, Y_val, verbose=1)

In [ ]:
# Plot the loss and accuracy curves for training and validation 
fig, ax = plt.subplots(2,1)
ax[0].plot(history.history['loss'], color='b', label="Training loss")
ax[0].plot(history.history['val_loss'], color='r', label="validation loss",axes =ax[0])
legend = ax[0].legend(loc='best', shadow=True)

ax[1].plot(history.history['accuracy'], color='b', label="Training accuracy")
ax[1].plot(history.history['val_accuracy'], color='r',label="Validation accuracy")
legend = ax[1].legend(loc='best', shadow=True)

In [ ]:
# predict results
results = model.predict(test)

# select the indix with the maximum probability
results = np.argmax(results,axis = 1)

results = pd.Series(results,name="Label")
results

In [ ]:
submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),results],axis = 1)
submission.to_csv("digit-recognizer.csv",index=False)